This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [145]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [146]:
# select model to train

model_inventory = {'mktprob': {'X_columns': ["mkt_prob"],
                                'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSig},
                   'mktprob_soft': {'X_columns': ["mkt_prob"],
                                'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSoft},
                    'mktprob_MLR': {'X_columns': ["mkt_prob"],
                                'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': MLR},
                   'AlunOwen_v0': {'X_columns': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v1': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v2': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinDropReluLinSoft},
                   'AlunOwen_v3': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR},
                   'AlunOwen_v4': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': [],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR}
                               }

model = 'AlunOwen_v3'
X_columns = model_inventory[model]['X_columns']
continuous_features = model_inventory[model]['continuous_features']
learning_rate = model_inventory[model]['learning_rate']
epochs = model_inventory[model]['epochs']
vacant_stall_indicator = model_inventory[model]['vacant_stall_indicator']
bias = model_inventory[model]['bias']
model_architecture = model_inventory[model]['model_architecture']

In [147]:
# read in data

y_columns = ["win"]

train_data_fn = "data\\runners_train.csv"
test_data_fn = "data\\runners_test.csv"

train_data = RacesDataset(train_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, continuous_features=continuous_features)
test_data = RacesDataset(test_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar, continuous_features=continuous_features)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [148]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.loc[:, train_data.X_columns].head()

age                                                    \
stall_number        1         2         3         4         5         6    
race_id                                                                    
11504        -0.661185 -0.661185 -0.661185 -0.661185 -0.661185 -0.661185   
11505        -0.130811  0.929938  2.521062  1.460313  0.399564  1.990687   
11506        -0.661185 -0.661185 -0.661185 -0.661185 -0.661185 -0.661185   
11507         0.399564 -0.130811 -0.130811 -0.130811 -0.130811 -0.130811   
11508         1.460313 -0.130811  0.399564 -0.130811  0.399564 -0.130811   

                                                                               \
stall_number        7         8         9         10        11   12   13   14   
race_id                                                                         
11504        -0.661185 -0.661185  0.000000  0.000000  0.000000  0.0  0.0  0.0   
11505         0.399564  0.399564  0.399564 -0.130811  0.399564  0.0  0.0  0.0   
11506        -0.661185 -0.661185 -0.661185 -0.661185  0.000000  0.0  0.0  0.0   
11507         1.990687 -0.130811 -0.130811  0.000000  0.000000  0.0  0.0  0.0   
11508         0.399564  0.399564 -0.130811  0.000000  0.000000  0.0  0.0  0.0   

                       blinkers                                               \
stall_number   15   16       1    2    3    4    5    6    7    8    9    10   
race_id                                                                        
11504         0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                           cheekpieces                      \
stall_number   11   12   13   14   15   16          1    2    3    4    5    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0  0.0   

                                                                      daysLTO  \
stall_number   6    7    8    9    10   11   12   13   14   15   16        1    
race_id                                                                         
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.634986   
11505         0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.634986   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.634986   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.634986   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.634986   

                                                                          \
stall_number        2         3         4         5         6         7    
race_id                                                                    
11504         1.634986  1.634986  1.634986  1.634986  1.634986  1.634986   
11505         1.634986  1.634986  1.634986  1.634986  1.634986  1.634986   
11506         1.634986  1.634986  1.634986  1.634986  1.634986  1.634986   
11507         1.634986  1.634986  1.634986  1.634986  1.634986  1.634986   
11508         1.634986  1.634986  1.634986  1.634986  1.634986  1.634986   

                                                                               \
stall_number        8         9         10        11   12   13   14   15   16   
race_id                                                                    

In [149]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]

# torch.manual_seed(0)
net = model_architecture(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: AlunOwen_v3
Layer: neural_network.0.weights | Size: torch.Size([20]) | Values : Parameter containing:
tensor([ 0.3004,  0.0711, -0.0148, -0.4675, -0.1407,  0.0822,  0.3056,  0.3288,
        -0.2628, -0.2755,  0.1167,  0.2750, -0.0122, -0.5232, -0.3148, -0.3000,
        -0.2004,  0.4862,  0.1423, -0.2923], requires_grad=True) 



In [150]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = net(X)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([0])
torch.Size([1, 320])


In [151]:
%env WANDB_NOTEBOOK_NAME 'C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "model": model,
    "X_columns": X_columns,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_stall_indicator": vacant_stall_indicator,
    "bias": bias,
    "model_architecture": list(net.modules())
    }
)

env: WANDB_NOTEBOOK_NAME='C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'


In [152]:
# optimizing model parameters

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, net, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

Epoch 1
-------------------------------
loss: 2.778336  [   64/16620]
loss: 2.759131  [ 6464/16620]
loss: 2.799328  [12864/16620]
Test Error: 
 Accuracy: 12.0%, Avg loss: 2.776081 

Epoch 2
-------------------------------
loss: 2.778444  [   64/16620]
loss: 2.756596  [ 6464/16620]
loss: 2.796352  [12864/16620]
Test Error: 
 Accuracy: 12.7%, Avg loss: 2.773848 

Epoch 3
-------------------------------
loss: 2.778503  [   64/16620]
loss: 2.753953  [ 6464/16620]
loss: 2.793190  [12864/16620]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.771457 

Epoch 4
-------------------------------
loss: 2.778481  [   64/16620]
loss: 2.751201  [ 6464/16620]
loss: 2.789817  [12864/16620]
Test Error: 
 Accuracy: 14.1%, Avg loss: 2.768888 

Epoch 5
-------------------------------
loss: 2.778334  [   64/16620]
loss: 2.748344  [ 6464/16620]
loss: 2.786205  [12864/16620]
Test Error: 
 Accuracy: 14.7%, Avg loss: 2.766117 

Epoch 6
-------------------------------
loss: 2.777999  [   64/16620]
loss: 2.745396  [ 64

In [153]:
# finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▂▃▄▅▆▇▇▇▇██▇███████████████████████████
loss,██▇▇▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.22533
loss,2.65835


In [154]:
for para_name, para_vals in net.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")